# Final Project
**LT Number: 5**

Names: Lacsam | Mendoza | Moran | Pascual

*Pusoy*, also known as Chinese Poker, is a 4-player card game widely played in the Philippines and other parts of Asia. Each player is dealt 13 cards and must arrange them into three poker hands: a 3-card front hand, a 5-card middle hand, and a 5-card back hand, in non-decreasing strength (i.e., back ≥ middle ≥ front). If this hierarchy is violated, the player automatically loses ("fouls"). After hands are submitted, each player compares their three hands against those of each opponent to compute scores. 

In [1]:
import time
import sys
import re
import traceback
import poker_stat
from card import Card, CARD_SUIT
from poker import Poker, LEFT_HAND_WINS, RIGHT_HAND_WINS, HANDS_ARE_EQUAL, \
                  ROYAL_FLUSH, STRAIGHT_FLUSH, FOUR_OF_A_KIND, FULL_HOUSE, \
                  FLUSH, STRAIGHT, THREE_OF_A_KIND, TWO_PAIR, ONE_PAIR, \
                  HIGH_CARD, HAND_TYPE_STRING

from test_card import test_card_class
from test_deck import test_deck_class
from test_player import test_player_class
from test_poker import test_poker_class

In [2]:
def print_welcome():
    """Display welcome banner and basic rules of Pusoy."""
    print("#" * 80)
    print("#                                                                              #")
    print("#                                  WELCOME TO                                  #")
    print("#                                                                              #")
    print("#           ########   ##      ##    ######      ######    #        #          #")
    print("#           ##     ##  ##      ##  ##      ##  ##      ##   #      #           #")
    print("#           ##     ##  ##      ##  ##          ##      ##    #    #            #")
    print("#           #######    ##      ##    ######    ##      ##     #  #             #")
    print("#           ##         ##      ##          ##  ##      ##      ##              #")
    print("#           ##         ##      ##  ##      ##  ##      ##      ##              #")
    print("#           ##           ######      ######      ######        ##              #")
    print("#                                                                              #")
    print("#                      LACSAM | MERCADO | MORAN | PASCUAL                      #")
    print("#                                                                              #")
    print("#" * 80)
    print("# RULES:                                                                       #")
    print("# Arrange 13 cards into 3 hands: Back (5), Middle (5), Front (3)               #")
    print("# Back hand must be strongest, Middle stronger than Front.                     #")
    print("#" * 80)

In [3]:
def show_hand_ranks():
    """Print the probabilities of differend hands in Pusoy using poker_stat."""
    print("\nPROBABILITIES OF POKER HANDS:\n")
    print(f"Royal Flush:     {100.0 * poker_stat.ROYAL_FLUSH_PROBABILITY:8.4f}%")
    print(f"Straight Flush:  {100.0 * poker_stat.STRAIGHT_FLUSH_PROBABILITY:8.4f}%")
    print(f"Four-of-a-Kind:  {100.0 * poker_stat.FOUR_OF_A_KIND_PROBABILITY:8.4f}%")
    print(f"Full House:      {100.0 * poker_stat.FULL_HOUSE_PROBABILITY:8.4f}%")
    print(f"Flush:           {100.0 * poker_stat.FLUSH_PROBABILITY:8.4f}%")
    print(f"Straight:        {100.0 * poker_stat.STRAIGHT_PROBABILITY:8.4f}%")
    print(f"Three-of-a-Kind: {100.0 * poker_stat.THREE_OF_A_KIND_PROBABILITY:8.4f}%")
    print(f"Two Pairs:       {100.0 * poker_stat.TWO_PAIR_PROBABILITY:8.4f}%")
    print(f"One Pair:        {100.0 * poker_stat.ONE_PAIR_PROBABILITY:8.4f}%")
    print(f"High Card:       {100.0 * poker_stat.HIGH_CARD_PROBABILITY:8.4f}%")

In [4]:
def show_possible_hands(cards):
    """List all possible poker hands that can be played given a set of 13 cards."""
    print("\nHINTS:\n")

    has_good_hand = False
    for i in range(ROYAL_FLUSH, HIGH_CARD, -1):
        mutable_hand = Poker.sort_hand_by_value(cards.copy())
        arranged_hand = []
        if Poker.extract_poker_hand(i, mutable_hand, arranged_hand):
            has_good_hand = True
            print(f"You have a {HAND_TYPE_STRING[i]}.")
    
    if not has_good_hand:
        print("You only have a high card!")        

In [5]:
def get_yes_no_input(prompt):
    """Get yes/no input from user
    
    Args:
        prompt (str): yes or no question to display to the user

    Returns:
        bool: True for 'y' or 'yes', False for 'n' or 'no'
    """
    while True:
        response = input(f"{prompt} (y/n): ").strip().lower()
        if response in ['yes', 'y']:
            return True
        elif response in ['no', 'n']:
            return False
        else:
            print("Please enter 'y' or 'n'.")

In [6]:
def display_cards(cards, title="Your cards"):
    """Display cards in a readable format with position index.
    
    Args:
        cards (list[Cards]) : List of Card objects to display
        title (str, optional) : Title to show above the cards. Default to "Your cards".

    Returns:
        None: Prints the layout of cards with front, middle, back hands and position indices.
    """
    print()
    print("                 ┌──FRONT───┐   ┌──────MIDDLE────────┐   ┌────────BACK────────┐")
    print(f"     {title}: ", end="")
    print("  ".join([f"{card.short_name():3s}" for card in cards]))
    print("       Position: 1    2    3    4    5    6    7    8    9    10   11   12   13")

In [7]:
def is_jupyter():
    """Check if the current environment is a Jupyter Notebook.

    Returns:
        bool: True if running inside a Jupyter Notebook, False otherwise.
    """
    try:
        from IPython import get_ipython
        return get_ipython() is not None and get_ipython().__class__.__name__ == 'ZMQInteractiveShell'
    except ImportError:
        return False

In [8]:
def clear_screen_lines(num_lines):
    """Clear the specified number of lines from terminal"""
    for _ in range(num_lines):
        sys.stdout.write('\033[1A')  # Move cursor up one line
        sys.stdout.write('\033[2K')  # Clear the line
    sys.stdout.flush()

In [9]:
def get_user_hand_arrangement(player, poker=None):
    """
    Get hand arrangement from user input using position indices.
        
    Allows the payer to rearrange 13 cards into 3 hands:
    Front (3), Middle (5), Back (5)

    User input options:
        - Enter 2 integers from 1 to 13 to swap positions of two cards
        - Enter 13 integers from 1 to 13 to specify desired card arrangement
        - Press ENTER to submit current arrangement
        - Enter 'A' to automatically arrange the cards
        - Enter 'h' to display hand rank probabilities

    Args:
        player (Player): The player object containing cards and arranged_cards attributes.
        poker (Poker, optional): The poker game object for auto-arrangement. Defaults to None.


    Returns:
        bool: True if card arrangment is successful and valid

    Notes:
        - Validates that positions are within 1-13 and contain no duplicates.
        - Updates `player.arranged_cards` with the final arrangement.
        - Calls `Poker.is_valid_hand` to verify the Pusoy rules for hand strength.    
    """
    # Display an error message, pause briefly, and clear the line
    def show_error(msg):
        print(msg)
        time.sleep(1)
        clear_screen_lines(1)
    
    print(f">>> {player.name}, arrange your 13 cards into 3 hands:")

    # Initialize with the original card order
    current_cards = player.cards.copy()

    while True:
        # Display current arrangement
        display_cards(current_cards)
    
        # Get user input for swapping
        user_input = input("\nEnter 2 numbers to swap, 13 numbers for full arrangement, or press ENTER to submit: ").strip()
        
        # Clear the input line
        if not is_jupyter():
            clear_screen_lines(1)

        # If empty input, user wants to submit
        if not user_input:
            if get_yes_no_input("Are you sure you want to submit your cards?"):
                break

        # If, secretly, user invokes auto arrange
        if user_input == 'A':
            poker.auto_arrange_for_player(player.player_id)
            current_cards = player.arranged_cards.copy()
            display_cards(current_cards)
            break

        if user_input == 'h':
            show_hand_ranks()
            continue

        if user_input == 'H':
            show_possible_hands(current_cards)
            continue

        # Parse the input
        try:
            positions = re.split(r"[ ,;:]", user_input) # user_input.split()
            pos_list = [int(p) for p in positions] #list of positional indices

            #if 2-integer input
            if len(pos_list) == 2:
                if len(positions) != 2:
                    show_error("Please enter exactly two position numbers separated by space.")
                    continue
    
                pos1 = int(positions[0])
                pos2 = int(positions[1])
    
                # Validate positions (1-13, convert to 0-12 for array indexing)
                if pos1 < 1 or pos1 > 13 or pos2 < 1 or pos2 > 13:
                    show_error("Positions must be between 1 and 13.")
                    continue
    
                if pos1 == pos2:
                    show_error("Please enter two different positions.")
                    continue
    
                # Convert to 0-based indexing and swap
                idx1 = pos1 - 1
                idx2 = pos2 - 1
                current_cards[idx1], current_cards[idx2] = current_cards[idx2], current_cards[idx1]
                # Clear the display to show updated arrangement
                clear_screen_lines(1)  # Clear the display_cards output

            #if 13-integer input
            elif len(pos_list) == 13:     
                if len(positions) != 13:
                    show_error("Please enter exactly 13 position numbers separated by space.")
                    continue
               
                # Validate positions (1-13, convert to 0-12 for array indexing)
                if any(p < 1 or p > 13 for p in pos_list):
                    show_error("Positions must be between 1 and 13.")
                    continue
    
                if len(set(pos_list)) != 13:
                    show_error("Duplicate positions detected. Please enter unique positions 1-13.")
                    continue
            
                # Convert to 0-based indexing and arrange
                idx = [p - 1 for p in pos_list]
                current_cards = [current_cards[i] for i in idx]
                # Clear the display to show updated arrangement
                clear_screen_lines(1)  # Clear the display_cards output

            else:
                show_error("Enter either 2 numbers (swap) or 13 numbers (arrange).")
                continue

        except ValueError:
            show_error("Please enter valid numbers.")
            continue

        except Exception as e:
            show_error(f"Error: {e}")
            continue

    # Set the final arrangement
    player.arranged_cards = current_cards

    # Extract hands for display
    front_cards = current_cards[:3]
    middle_cards = current_cards[3:8]
    back_cards = current_cards[8:13]

    # Validate the hand
    if not Poker.is_valid_hand(player.arranged_cards):
        print("\n⚠️  WARNING: Your hand arrangement is invalid!")
        print("Back hand must be stronger than Middle, Middle stronger than Front")
        print("Current arrangement violates Pusoy rules!")
        if get_yes_no_input("Do you want to rearrange"):
            return get_user_hand_arrangement(player)

    return True

In [10]:
def display_all_hands(poker):
    """
    Display all players' arranged hands

    This shows the player's Front, Middle, and Back Hands 
    
    Args:
        poker (Poker): An instance of the Poker game containing player data,
            including arranged cards and methods to analyze hand types.

    Returns:
        None: Prints output directly to the console.
    """

    print("\n" + "=" * 80)
    print("🎯 FINAL HANDS")
    print("")

    print("                 ┌──FRONT───┐   ┌──────MIDDLE────────┐   ┌────────BACK────────┐")
    for i, player in enumerate(poker.players):
        print(f"{player.name:15s}: ", end="")
        print("  ".join([f"{card.short_name():3s}" for card in player.arranged_cards]))

        # Analyze each hand
        front_info = Poker.analyze_hand(player.arranged_cards[0:3])
        middle_info = Poker.analyze_hand(player.arranged_cards[3:8])
        back_info = Poker.analyze_hand(player.arranged_cards[8:13])

        print(f"{" "*17}{Poker.get_hand_type_string(front_info.hand_type):13s}  "
              f"{Poker.get_hand_type_string(middle_info.hand_type):23s}  "
              f"{Poker.get_hand_type_string(back_info.hand_type)}")

In [11]:
def display_results(poker):
    """
    Display game results and scores for all players in Pusoy.

    This shows player scores, ranks the players, and prints a
    formatted summary of each player's performance for the Front, Middle,
    and Back hands. It also indicates ties or the winner.
    
    Args:
        poker (Poker): An instance of the Poker game containing player data
            and methods to update scores and rank players.
    
    Returns:
        None: Prints output directly to the console.
    """
    print("\n" + "=" * 80)
    print("🏆 GAME RESULTS")
    print("")

    # Update scores
    poker.update_player_scores()

    # Sort players by score
    ranked_players = poker.players_ranked_by_score()

    # Display individual hand comparisons
    hand_names = ["Front", "Middle", "Back"]

    print("                 Front──────   Middle─────   Back───────   SCORE")
    print("    VS PLAYER>>  ─1──2──3──4   ─1──2──3──4   ─1──2──3──4")
    for i, player in enumerate(ranked_players):
        print(f"{player.name:15s}: ", end="")
        results = ""
        for hand_idx in range(3):
            for j, opponent in enumerate(poker.players):
                if player.player_id == opponent.player_id:
                    results += "-- "
                elif player.raw_scores[j][hand_idx] > 0:
                    results += "+1 "
                elif player.raw_scores[j][hand_idx] < 0:
                    results += "-1 "
                else:
                    results += "+0 "
            results += "  "
        results += f"{player.score:2d}"
        if i == 3:
            if player.score == ranked_players[2].score:
                results += " TIE"
            else:
                results += " WINNER"
        print(results)

In [12]:
def play_single_game(play_with_bots=True):
    """
    Play a single game of Pusoy.

    This initializes a new game, shuffles and distributes
    cards, allows players (human or bots) to arrange their hands, and
    then displays all hands and the game results.

    The function supports playing against bots or all human players:
        - If `play_with_bots` is True, the human arranges their hand and
          the bot players are automatically arranged using different algorithms.
        - If False, all players manually arrange their hands.

    Args:
        play_with_bots (bool, optional): Whether to play with bot players.
            Defaults to True.

    Returns:
        None: Prints game progress, hands, and results
        directly to the console.
    """
    # Create poker game and seed with current time
    poker = Poker()
    seed = int(time.time()) - 1755567090 # Just so that the number is low.
    print("=" * 80)
    print(f"Starting game #{seed}...")

    # Shuffle and distribute cards
    poker.shuffle_deck(seed)
    poker.distribute_cards()
    poker.distribute_hands()

    print(f"Cards distributed to all players!")

    if play_with_bots:
        # Human player arranges cards
        human_player = poker.this_player()
        if not get_user_hand_arrangement(human_player, poker):
            print("Failed to arrange cards. Ending game.")
            return

        # Auto-arrange for bot players
        for i in range(1, 4):  # Players 1, 2, 3 are bots
            bot_player = poker.players[i]
            if i == 3:
                poker.auto_arrange_for_player(i, algo='balanced')
            else:
                poker.auto_arrange_for_player(i, algo='greedy')

    else:
        for i in range(0, 4):
            player = poker.players[i]
            print("")
            if not get_user_hand_arrangement(player, poker):
                print("Failed to arrange cards. Ending game.")
                return

    # Display all hands
    display_all_hands(poker)

    # Display results
    display_results(poker)

In [ ]:
# """Main game loop"""
print_welcome()

# Ask if player wants to play
selection = '0'
while selection not in ['1','2','3']:
    selection = input("Choose from the following:\n(1) Human vs Human\n(2) Human vs Bots\n(3) Quit\n").strip()
    if selection == '3':
        print("Goodbye!")
        break

print("Instructions:")
print("- Arrange cards by swapping positions, or by providing the full sequence.")
print("- Enter two position numbers (1-13) separated by space to swap cards.")
print("- Or enter 13 position numbers to specify the full sequence of cards.")
print("- Press Enter without typing anything when you're ready to submit.")
print("- Press 'h' to display the different types of poker hands.")
print("- Press 'H' to display the list of possible poker hands given your 13 cards.")
print("- Positions 1-3: FRONT hand, 4-8: MIDDLE hand, 9-13: BACK hand")
print("- Remember: BACK must be strongest, MIDDLE stronger than FRONT")
print("")

while True:
    try:
        # Play a single game
        play_single_game(selection=='2')

    except KeyboardInterrupt:
        print("\n\n⚠️  Game interrupted by user.")
        if not get_yes_no_input("Do you want to continue playing"):
            break

    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        traceback.print_exc()
        print("Please try again.")

    # Ask if player wants to play another game
    if not get_yes_no_input("\nDo you want to play another game?"):
        print("\n👋 Thanks for playing! Goodbye!")
        break

## Asserts
Run the following cell if you wish to test all custom classes used by this application.

In [13]:
test_card_class()
test_deck_class()
test_player_class()
test_poker_class()

All Card tests passed!
All Deck tests passed!
All Player tests passed ✅
Pack 0 dump:
07 of clubs
03 of diamonds
02 of hearts
13 of diamonds
04 of diamonds
05 of diamonds
12 of diamonds
07 of hearts
05 of clubs
01 of spades
02 of diamonds
05 of hearts
13 of hearts
Player 0 dump:
07 of clubs
03 of diamonds
02 of hearts
13 of diamonds
04 of diamonds
05 of diamonds
12 of diamonds
07 of hearts
05 of clubs
01 of spades
02 of diamonds
05 of hearts
13 of hearts
Player1>>
7♣ 3♦ 2♥ K♦ 4♦ 5♦ Q♦ 7♥ 5♣ A♠ 2♦ 5♥ K♥
Player2>>
2♠ 6♦ 8♦ J♠ 7♦ K♣ 9♥ 3♣ 4♥ 2♣ 6♥ 6♣ 7♠
Player3>>
K♠ 8♠ J♣ 4♣ 3♠ 10♣ 9♠ 10♥ Q♥ 6♠ 5♠ J♥ Q♣
Player4>>
10♠ A♥ A♣ 4♠ 9♦ J♦ Q♠ A♦ 10♦ 9♣ 8♣ 3♥ 8♥
All Poker tests passed!


## Testing a Specific Game
Run the following to test a specific Game No (seed).

In [14]:
def test_game(game_no, strategy='greedy'):
    print(f"Test Game No. {game_no}")
    poker = Poker()

    # Shuffle and distribute cards
    poker.shuffle_deck(game_no)
    poker.distribute_cards()
    poker.distribute_hands()

    # Auto-arrange for bot players
    for i in range(4):
        poker.auto_arrange_for_player(i, algo=strategy)

    # Display all hands
    display_all_hands(poker)

    # Display results
    display_results(poker)

In [15]:
# In game 705805, Player 1 gets a straight flush and a four-of-a-kind! How lucky!
test_game(705805)

Test Game No. 705805

🎯 FINAL HANDS

                 ┌──FRONT───┐   ┌──────MIDDLE────────┐   ┌────────BACK────────┐
Player1        : 7♠   8♦   Q♠   10♦  10♠  10♣  10♥  5♥   5♣   6♣   7♣   8♣   9♣ 
                 High card      Four-of-a-kind           Straight flush
Player2        : 4♠   7♥   K♠   2♠   2♦   J♠   J♥   A♣   4♦   6♦   7♦   9♦   K♦ 
                 High card      Two pairs                Flush
Player3        : 4♥   6♠   J♦   8♥   8♠   9♥   9♠   3♣   A♦   A♥   A♠   2♥   2♣ 
                 High card      Two pairs                Full house
Player4        : 4♣   6♥   J♣   5♠   5♦   K♣   K♥   3♠   Q♣   Q♥   Q♦   3♥   3♦ 
                 High card      Two pairs                Full house

🏆 GAME RESULTS

                 Front──────   Middle─────   Back───────   SCORE
    VS PLAYER>>  ─1──2──3──4   ─1──2──3──4   ─1──2──3──4
Player3        : -1 -1 -- +0   -1 -1 -- -1   -1 +1 -- +1   -4
Player4        : -1 -1 +0 --   -1 +1 +1 --   -1 +1 -1 --   -2
Player2        : +1 -- +1

In [16]:
def find_invalid_hand(start=0, end=1_000, strategy='greedy'):
    for game_no in range(start, end):
        poker = Poker()
    
        # Shuffle and distribute cards
        poker.shuffle_deck(game_no)
        poker.distribute_cards()
        poker.distribute_hands()
    
        # Auto-arrange for bot players
        for i in range(4):
            poker.auto_arrange_for_player(i, algo=strategy)

        # Test for invalid hands
        for i, player in enumerate(poker.players):
            if not Poker.is_valid_hand(player.arranged_cards):
                print(f"Invalid hand found in game {game_no}, player {i + 1}")
                display_all_hands(poker)

In [19]:
find_invalid_hand(end=10_000, strategy='greedy')
#find_invalid_hand(end=10_000, strategy='balanced')